In [38]:
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
warnings.filterwarnings("ignore", category=FutureWarning) #ignore warnings

In [2]:
df = pd.read_excel('forecasting_case_study.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Product             1218 non-null   object        
 1   date                1218 non-null   datetime64[ns]
 2   Sales               1218 non-null   int64         
 3   Price Discount (%)  1218 non-null   float64       
 4   In-Store Promo      1218 non-null   int64         
 5   Catalogue Promo     1218 non-null   int64         
 6   Store End Promo     1218 non-null   int64         
 7   Google_Mobility     1218 non-null   float64       
 8   Covid_Flag          1218 non-null   int64         
 9   V_DAY               1218 non-null   int64         
 10  EASTER              1218 non-null   int64         
 11  CHRISTMAS           1218 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(8), object(1)
memory usage: 114.3+ KB


In [5]:
df.head()

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2017-02-05,27750,0.00,0,0,0,0.0,0,0,0,0
1,SKU1,2017-02-12,29023,0.00,1,0,1,0.0,0,1,0,0
2,SKU1,2017-02-19,45630,0.17,0,0,0,0.0,0,0,0,0
3,SKU1,2017-02-26,26789,0.00,1,0,1,0.0,0,0,0,0
4,SKU1,2017-03-05,41999,0.17,0,0,0,0.0,0,0,0,0


In [7]:
df.isnull().sum().sum()

0

There are no missing values in the data set.

# Data Understanding

In [49]:
unique_products = df["Product"].unique()
product_counts = df['Product'].value_counts()
print(f"There are {len(unique_products)} total unique products in the data set. \nThe product SKU numbers are listed below with the item count next to them:")
print(product_counts)

There are 6 total unique products in the data set. 
The product SKU numbers are listed below with the item count next to them:
SKU1    204
SKU2    204
SKU3    204
SKU4    204
SKU5    204
SKU6    198
Name: Product, dtype: int64


In [24]:
early = df["date"].min()
late = df["date"].max()
print(f"The earliest date in this data set is {early} and the latest date is {late}.")

The earliest date in this data set is 2017-02-05 00:00:00 and the latest date is 2020-12-27 00:00:00.


In [30]:
df.describe()

,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
count,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000
mean,30294.678982,0.251043,0.472085,0.212644,0.348933,-2.377406,0.226601,0.019704,0.019704,0.018883
std,35032.527297,0.215494,0.499425,0.409346,0.476828,5.806291,0.418804,0.139040,0.139040,0.136169
min,0.000000,0.000000,0.000000,0.000000,0.000000,-28.490000,0.000000,0.000000,0.000000,0.000000
25%,7212.750000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,19742.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,40282.250000,0.400000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,288322.000000,0.830000,1.000000,1.000000,1.000000,3.900000,1.000000,1.000000,1.000000,1.000000


In [35]:
labels = df.columns.values.tolist()
rs = np.random.RandomState(0)
pd.DataFrame(rs.rand(12, 12), index = labels,
                  columns = labels)
corr = df.corr()
#show correlation heatmap plot with 2 digit precision
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

#code to mask upper half of the correlational matrix
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
Sales,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Price Discount (%),0.40,nan,nan,nan,nan,nan,nan,nan,nan,nan
In-Store Promo,0.24,0.22,nan,nan,nan,nan,nan,nan,nan,nan
Catalogue Promo,-0.12,-0.07,-0.49,nan,nan,nan,nan,nan,nan,nan
Store End Promo,0.24,0.22,0.37,0.12,nan,nan,nan,nan,nan,nan
Google_Mobility,0.06,-0.21,0.06,0.07,0.08,nan,nan,nan,nan,nan
Covid_Flag,-0.09,0.28,-0.02,-0.09,-0.08,-0.76,nan,nan,nan,nan
V_DAY,-0.01,-0.04,0.02,-0.04,0.02,0.08,0.01,nan,nan,nan
EASTER,-0.01,0.00,0.02,-0.04,-0.07,-0.11,0.01,-0.02,nan,nan
CHRISTMAS,-0.04,-0.01,0.01,0.05,-0.00,0.00,-0.00,-0.02,-0.02,nan


Identify data points outside of 3 standard deviations:

In [52]:
mean_sales = df['Sales'].mean()
std_sales = df['Sales'].std()
potential_outliers = df.loc[(df['Sales'] > mean_sales + 3*std_sales) | (df['Sales'] < mean_sales - 3*std_sales)]
print(potential_outliers.count())
print(potential_outliers)

Product               26
date                  26
Sales                 26
Price Discount (%)    26
In-Store Promo        26
Catalogue Promo       26
Store End Promo       26
Google_Mobility       26
Covid_Flag            26
V_DAY                 26
EASTER                26
CHRISTMAS             26
dtype: int64
     Product       date   Sales  Price Discount (%)  In-Store Promo  \
46      SKU1 2017-12-24  155253                0.17               1   
98      SKU1 2018-12-23  174994                0.28               1   
150     SKU1 2019-12-22  258874                0.44               0   
421     SKU3 2017-05-07  212245                0.50               1   
426     SKU3 2017-06-11  248373                0.50               1   
437     SKU3 2017-08-27  236242                0.50               1   
447     SKU3 2017-11-05  212947                0.50               1   
478     SKU3 2018-06-10  288322                0.50               1   
494     SKU3 2018-09-30  237462                0

In [54]:
df.shape[0]

1218

Since there are only 26 data points out of 1218, this suggests that the data is relatively clean and does not have many extreme values.